# Fine-Tuning the CNN on Cancer and Subtype





In [1]:
import os
import sys
from pathlib import Path

import pandas as p
import numpy as np

import tensorflow as tf
from tensorflow import keras
#Add the parent directory to access ENV variables
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#Import of necessary paths ( GDC data Path and Dataset folder)
from config import THYROID_PATH, MODEL_PATH, RESULTS_PATH

2025-04-11 15:43:13.345648: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 15:43:13.396225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-11 15:43:13.396281: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-11 15:43:13.396292: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 15:43:13.404241: I tensorflow/core/platform/cpu_feature_g

In [2]:
FinalPath = Path(THYROID_PATH,'FinalData.npy')
npzfiles = np.load(FinalPath,allow_pickle=True)


#output_folder = Path(RESULTS_PATH,'UnfilteredCancer/')
#output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [3]:
def get_uncompiled_model(reset_last_layer=False):
    
    model = keras.models.load_model(os.path.join(MODEL_PATH,'pan-cancer-leaky-relu'))
    if(reset_last_layer):
        output_follicolar= keras.layers.Dense(1, activation='sigmoid',name='output_follicolar')(model.layers[-2].output)
        model = keras.models.Model(inputs=model.input, outputs = [output_follicolar])
    return model

def get_compiled_model(metrics=None,reset_last_layer=False):
    
    if(metrics is None):
        metrics = [
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.AUC(name='auc'),
              keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        ]
        
    model = get_uncompiled_model(reset_last_layer)
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(3e-5),metrics=metrics)
    
    return model

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

def make_or_restore_model(checkpoint_dir = "ckpt", reset_last_layer=False):
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model(reset_last_layer=reset_last_layer)

In [4]:
#Retrain Base Model for thyroid cancer classification

In [ ]:
X_train = np.nan_to_num(npzfiles['X_cancer_train'])
y_train = npzfiles['y_cancer_train']

model = make_or_restore_model('ckpt-thyroid-cancer')

cancer_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)

cancer_df = pd.DataFrame(cancer_history.history)
cancer_df.to_csv("fit_history/thyroid-cancer.csv")
model.save(os.path.join(MODEL_PATH,"thyroid-cancer"))

Creating a new model


2025-04-11 15:43:27.089981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-11 15:43:27.135443: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-11 15:43:27.135781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/30


2025-04-11 15:43:39.961772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2025-04-11 15:43:43.306068: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f93d80b8950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-11 15:43:43.306127: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2025-04-11 15:43:43.311621: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-11 15:43:43.440842: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


42/42 [==============================] - 43s 647ms/step - loss: 4.4613 - tp: 259.0000 - fp: 5.0000 - tn: 25.0000 - fn: 40.0000 - accuracy: 0.8632 - precision: 0.9811 - recall: 0.8662 - auc: 0.8690 - prc: 0.9792 - val_loss: 1.1587 - val_tp: 67.0000 - val_fp: 2.0000 - val_tn: 12.0000 - val_fn: 2.0000 - val_accuracy: 0.9518 - val_precision: 0.9710 - val_recall: 0.9710 - val_auc: 0.9099 - val_prc: 0.9677
Epoch 2/30
42/42 [==============================] - 25s 596ms/step - loss: 2.4494 - tp: 286.0000 - fp: 10.0000 - tn: 20.0000 - fn: 13.0000 - accuracy: 0.9301 - precision: 0.9662 - recall: 0.9565 - auc: 0.8431 - prc: 0.9709 - val_loss: 1.3007 - val_tp: 67.0000 - val_fp: 4.0000 - val_tn: 10.0000 - val_fn: 2.0000 - val_accuracy: 0.9277 - val_precision: 0.9437 - val_recall: 0.9710 - val_auc: 0.8478 - val_prc: 0.9428
Epoch 3/30
42/42 [==============================] - 25s 591ms/step - loss: 2.0094 - tp: 288.0000 - fp: 9.0000 - tn: 21.0000 - fn: 11.0000 - accuracy: 0.9392 - precision: 0.9697 - r

In [ ]:
X_train = np.nan_to_num(npzfiles['X_subtype_train'])
y_train = npzfiles['y_subtype_train']

model = make_or_restore_model('ckpt-thyroid-subtype', reset_last_layer=True)

subtype_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)
subtype_df = pd.DataFrame(subtype_history.history)
subtype_df.to_csv(os.path.join('fit_history','thyroid-subtype.csv'))

model.save(os.path.join(MODEL_PATH,"thyroid-subtype"))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.2,shuffle=True, stratify=y)

from collections import Counter
X_train_fill = SimpleImputer(strategy='constant').fit_transform(X_train,y_train)
X_train_over,y_train_over = over.fit_resample(X_train_fill,y_train)
X_train_under,y_train_under = under.fit_resample(X_train_over,y_train_over)

X_test_fill = SimpleImputer(strategy='constant').fit_transform(X_test,y_test)
X_test_over,y_test_over = over.fit_resample(X_test_fill,y_test)
X_test_under,y_test_under = under.fit_resample(X_test_over,y_test_over)

print(Counter(y_train))
print(Counter(y_train_under))

print(Counter(y_test))
print(Counter(y_test_under))

